In [15]:
import pdfminer
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams

from pdfminer.pdfdevice import PDFDevice

from io import StringIO

import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd 
from pdf2image import convert_from_path

fname = '1909.12744.pdf'
fp = open(fname, 'rb')

In [16]:
parser = PDFParser(fp)
password = ''
document = PDFDocument(parser, password)
retstr = StringIO()
if not document.is_extractable:
    raise PDFTextExtractionNotAllowed
rsrcmgr = PDFResourceManager()

In [17]:
layout_type = ['LTTextBox', 'LTFigure', 'LTImage', 'LTCurve', 'LTRect']
color = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (160, 32, 240)]

draw_color = dict(zip(layout_type, color))

In [18]:
def parse_obj(lt_objs):

    boxs = {x: [] for x in layout_type}
    # loop over the object list
    for obj in lt_objs:

        if isinstance(obj, pdfminer.layout.LTTextBoxHorizontal):
            boxs['LTTextBox'].append(obj.bbox)
#         elif isinstance(obj, pdfminer.layout.LTFigure):
#             boxs['LTFigure'].append(obj.bbox)
#         elif isinstance(obj, pdfminer.layout.LTImage):
#             boxs['LTImage'].append(obj.bbox)
#         elif isinstance(obj, pdfminer.layout.LTCurve):
#             boxs['LTCurve'].append(obj.bbox)
#         elif isinstance(obj, pdfminer.layout.LTRect):
#             boxs['LTRect'].append(obj.bbox)
        else:
            pass
            #raise
    return boxs

In [19]:
def text_info_extract(objcet, parms):
    if len(parms) ==2:
        x,y = parms
        Lic = list()
        height = layout.bbox[3]
        for num,obj in enumerate(objcet):
            x_0 = obj.bbox[0] 
            if x_0 > x-10 and y-10 > x_0 : 
                x_0,y_0,x_1,y_1= obj.bbox
                box = (x_0, height-y_1, x_1, height-y_0)
                box_integer = tuple([round(jj) for jj in obj.bbox])
                txt = obj.get_text().replace('\n', '')
                x1,y1,x2,y2 = box_integer
                Lic.append((x1,y1,x2,y2,txt))
        return Lic

In [20]:
fname = '1909.12744.pdf'
fp = open(fname, 'rb')
parser = PDFParser(fp)
password = ''
document = PDFDocument(parser, password)
retstr = StringIO()

if not document.is_extractable:
    raise PDFTextExtractionNotAllowed
rsrcmgr = PDFResourceManager()

layout_type = ['LTTextBox', 'LTFigure', 'LTImage', 'LTCurve', 'LTRect']
color = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (160, 32, 240)]
draw_color = dict(zip(layout_type, color))


def parse_obj(lt_objs):
    
    boxs = {x: [] for x in layout_type}
    for obj in lt_objs:

        if isinstance(obj, pdfminer.layout.LTTextBoxHorizontal):
            boxs['LTTextBox'].append(obj.bbox)
#         elif isinstance(obj, pdfminer.layout.LTFigure):
#             boxs['LTFigure'].append(obj.bbox)
#         elif isinstance(obj, pdfminer.layout.LTImage):
#             boxs['LTImage'].append(obj.bbox)
#         elif isinstance(obj, pdfminer.layout.LTCurve):
#             boxs['LTCurve'].append(obj.bbox)
#         elif isinstance(obj, pdfminer.layout.LTRect):
#             boxs['LTRect'].append(obj.bbox)
        else:
            pass
            #raise
    return boxs


image = convert_from_path(fname)

# Set parameters for analysis.
laparams = LAParams()
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)

page_boxs = []
for page in PDFPage.create_pages(document):
    interpreter.process_page(page)
    
    # receive the LTPage object for the page.
    layout = device.get_result()
    boxs = parse_obj(layout._objs)
    page_sized = tuple([round(i) for i in layout.bbox])
    height = page_sized[-1]
    page_boxs.append((page_sized, boxs))
    page_info = text_info_extract(layout,(72,307))
    
    break

# page_info

In [21]:
def issue_area_replace(page_info):
    # 데이터 정렬 
    sample = np.array(page_info).reshape(-1,5)
    df = pd.DataFrame(sample,columns=['x1','y1','x2','y2','text'])
    df = df.astype({'x1':'int','y1':'int','x2':'int','y2':'int','text':'str'})
    df = df.sort_values(by=['y1','y2','x1'],ascending=(False,False,True))
    
    # text replace 
    stmp = 0
    txt = list()
    overlap_key = df['y1'].value_counts()[df['y1'].value_counts().values > 1].keys()

    for idx, val in zip(df['text'].index, df['y1']):
        if val in overlap_key:
            if stmp != val:
                stmp = val
                txt_smb = df[df['y1'] ==val]['text'].tolist()
                join_txt = " ".join(txt_smb)
                txt.append(join_txt)
            else:pass
        else:
            txt.append(df['text'][idx])
    
    return txt

text_list = issue_area_replace(page_info)
replace_text = "\n".join(text_list)
print(replace_text)

On the use of BERT for Neural Machine Translation
St´ephane ClinchantNAVER LABS Europe, Francestephane.clinchant@naverlabs.com
Kweon Woo JungNAVER Corp.,South Koreakweonwoo.jung@navercorp.com
Abstract
Exploiting large pretrained models for var-ious NMT tasks have gained a lot of vis-In this work we studyibility recently.how BERT pretrained models could be ex-ploited for supervised Neural Machine Trans-lation. We compare various ways to in-tegrate pretrained BERT model with NMTmodel and study the impact of the mono-lingual data used for BERT training on theﬁnal translation quality. We use WMT-14English-German, IWSLT15 English-Germanand IWSLT14 English-Russian datasets forthese experiments. In addition to standard tasktest set evaluation, we perform evaluation onout-of-domain test sets and noise injected testsets, in order to assess how BERT pretrainedrepresentations affect model robustness.
1 Introduction
Pretrained Language Models (LM) such as ELMOand BERT (Peters et al., 2018; Devlin 

In [22]:
sample = np.array(page_info).reshape(-1,5)
df = pd.DataFrame(sample,columns=['x1','y1','x2','y2','text'])
df = df.astype({'x1':'int','y1':'int','x2':'int','y2':'int','text':'str'})
df = df.sort_values(by=['y2','y1'],ascending=False)
df

,x1,y1,x2,y2,text
0,138,757,460,772,On the use of BERT for Neural Machine Translation
1,65,688,237,727,"St´ephane ClinchantNAVER LABS Europe, Francest..."
2,226,674,371,727,"Kweon Woo JungNAVER Corp.,South Koreakweonwoo...."
3,159,606,203,618,Abstract
4,89,393,273,594,Exploiting large pretrained models for var-iou...
5,72,368,78,380,1
6,90,368,155,380,Introduction
7,72,131,290,358,Pretrained Language Models (LM) such as ELMOan...
8,158,117,171,128,the
9,181,117,228,128,results of


# 텍스트 추출

In [23]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

output_string = StringIO()
with open(fname, 'rb') as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)

print(output_string.getvalue())

On the use of BERT for Neural Machine Translation

St´ephane Clinchant
NAVER LABS Europe, France
stephane.clinchant@naverlabs.com

Kweon Woo Jung
NAVER Corp.,
South Korea
kweonwoo.jung@navercorp.com

Vassilina Nikoulina
NAVER LABS Europe, France
vassilina.nikoulina@naverlabs.com

9
1
0
2

p
e
S
7
2

]
L
C
.
s
c
[

1
v
4
4
7
2
1
.
9
0
9
1
:
v
i
X
r
a

Abstract

Exploiting large pretrained models for var-
ious NMT tasks have gained a lot of vis-
In this work we study
ibility recently.
how BERT pretrained models could be ex-
ploited for supervised Neural Machine Trans-
lation. We compare various ways to in-
tegrate pretrained BERT model with NMT
model and study the impact of the mono-
lingual data used for BERT training on the
ﬁnal translation quality. We use WMT-14
English-German, IWSLT15 English-German
and IWSLT14 English-Russian datasets for
these experiments. In addition to standard task
test set evaluation, we perform evaluation on
out-of-domain test sets and noise injected test
sets

---

In [30]:
import pdfminer
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams

from pdfminer.pdfdevice import PDFDevice

from io import StringIO

import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd 
from pdf2image import convert_from_path

fname = '1909.12744.pdf'
fp = open(fname, 'rb')

In [32]:
def parse_obj(lt_objs, height):
    #     print(lt_objs)
    Lic = list()
#     height = obj.bbox[3]
    for obj in lt_objs:
        if isinstance(obj, pdfminer.layout.LTTextBoxHorizontal):
            
            x_0 = obj.bbox[0] 
            x_0,y_0,x_1,y_1= obj.bbox
            box = (x_0, height-y_1, x_1, height-y_0)
            box_integer = tuple([round(jj) for jj in obj.bbox])
            txt = obj.get_text().replace('\n', '')
            x1,y1,x2,y2 = box_integer
            Lic.append((x1,y1,x2,y2,txt))
            
    return Lic

def issue_area_replace(page_info):
    # 데이터 정렬 
    sample = np.array(page_info).reshape(-1,5)
    df = pd.DataFrame(sample,columns=['x1','y1','x2','y2','text'])
    df = df.astype({'x1':'int','y1':'int','x2':'int','y2':'int','text':'str'})
    df = df.sort_values(by=['y1','y2','x1'],ascending=(False,False,True))
    
    # text replace 
    stmp = 0
    txt = list()
    overlap_key = df['y1'].value_counts()[df['y1'].value_counts().values > 1].keys()

    for idx, val in zip(df['text'].index, df['y1']):
        if val in overlap_key:
            if stmp != val:
                stmp = val
                txt_smb = df[df['y1'] ==val]['text'].tolist()
                join_txt = " ".join(txt_smb)
                txt.append(join_txt)
            else:pass
        else:
            txt.append(df['text'][idx])
    
    return txt

In [33]:

fname = '1909.12744.pdf'
fp = open(fname, 'rb')

# 여기서 부터가 진짜
parser = PDFParser(fp)
document = PDFDocument(parser)
rsrcmgr = PDFResourceManager()
device = PDFDevice(rsrcmgr)
laparams = LAParams()
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)

p_box = list()
num = 1
for page in PDFPage.create_pages(document):
    print('Page:', num)
    interpreter.process_page(page)
    layout = device.get_result()
    height = layout.bbox[3]
    page_info = parse_obj(layout._objs, height)
    text_list = issue_area_replace(page_info)
    replace_text = "\n".join(text_list)
#     print(replace_text)
    p_box.append((num,replace_text))
    num += 1
fp.close()


Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10


In [34]:
for i,j in p_box:
    print( )
    print('page',i)
    print( )
    print(j)


page 1

On the use of BERT for Neural Machine Translation
St´ephane ClinchantNAVER LABS Europe, Francestephane.clinchant@naverlabs.com Vassilina NikoulinaNAVER LABS Europe, Francevassilina.nikoulina@naverlabs.com
Kweon Woo JungNAVER Corp.,South Koreakweonwoo.jung@navercorp.com
Abstract
mostly shown improvements in low-resource orunsupervised NMT settings, and has been littlestudied in standard supervised scenario with rea-sonable amount of data available.
9102
peS72
]LC.sc[
Exploiting large pretrained models for var-ious NMT tasks have gained a lot of vis-In this work we studyibility recently.how BERT pretrained models could be ex-ploited for supervised Neural Machine Trans-lation. We compare various ways to in-tegrate pretrained BERT model with NMTmodel and study the impact of the mono-lingual data used for BERT training on theﬁnal translation quality. We use WMT-14English-German, IWSLT15 English-Germanand IWSLT14 English-Russian datasets forthese experiments. In addition to standard

In [ ]:
# 좌우 워터 마크 제거
def parse_obj(lt_objs, b_size):
    Lic = list()
    a, b, c, d = b_size
    add_integer = tuple([round(jj) for jj in (a+50,b+50,c-50,d-50)])
    
    for obj in lt_objs:
        if obj.bbox[0] >add_integer[0]  and obj.bbox[2] >add_integer[0] and obj.bbox[1] <add_integer[-1] and obj.bbox[3] <add_integer[-1]  : 
            if isinstance(obj, pdfminer.layout.LTTextBoxHorizontal):

                x_0 = obj.bbox[0] 
                x_0,y_0,x_1,y_1= obj.bbox
                box = (x_0, d-y_1, x_1, d-y_0)
                box_integer = tuple([round(jj) for jj in obj.bbox])
                txt = obj.get_text().replace('\n', '')
                x1,y1,x2,y2 = box_integer
                Lic.append((x1,y1,x2,y2,txt))
            
    return Lic